<a href="https://colab.research.google.com/github/oenereaty/MP2024/blob/main/MP2024_wk10_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
#!pip uninstall mp2024pkg -y
!pip install git+https://github.com/guebin/mp2024pkg.git

  Cloning https://github.com/guebin/mp2024pkg.git to /tmp/pip-req-build-nbiciulh
  Running command git clone --filter=blob:none --quiet https://github.com/guebin/mp2024pkg.git /tmp/pip-req-build-nbiciulh
  Resolved https://github.com/guebin/mp2024pkg.git to commit 8667c802e9e2e3d3219a9fb952b9ab0a47f7e08d
  Preparing metadata (setup.py) ... done
  Created wheel for mp2024pkg: filename=mp2024pkg-1.0-py3-none-any.whl size=4614 sha256=ecf6247aecd4d68f3cce272f248d542f7b814c09f93dbfc454d1e1eb0784fe8b
  Stored in directory: /tmp/pip-ephem-wheel-cache-rjsh9y1x/wheels/34/e6/bd/d7e7b36b46f974e90ffd4f6308065d303cde840f9c0e0b0892
Successfully built mp2024pkg


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import datasets
import transformers
import torch
from mp2024pkg import signature, show
from rich import print as rprint

In [4]:
emotion = datasets.load_dataset('emotion')
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Dataset 형식 이해

In [5]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [6]:
d.to_list()

[{'text': 'i didnt feel humiliated', 'label': 0},
 {'text': 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'label': 0},
 {'text': 'im grabbing a minute to post i feel greedy wrong', 'label': 3},
 {'text': 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'label': 2}]

In [7]:
d['text']

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property']

In [8]:
d['label']

[0, 0, 3, 2]

`-` 위의 결과를 관찰하면 `Dataset` 는 마치 `dictionary` 처럼 느껴진다. 실제로 경우에 따라서 `Dataset`을 `dictionary` 처럼 생각해도 된다. `이때 Dataset`은 아래와 같은 구조로 이해하는게 편리하다.

- d = examples = {‘text’:[xxx,xxxx,xxxxx,xxxxxx], ‘label’:[yyy,yyyy,yyyyy,yyyyyy]}


In [9]:
dct = {'text': d['text'], 'label':d['label']}
dct

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property'],
 'label': [0, 0, 3, 2]}

- 복습: `dictionary`를 `data frame`과 비슷하게 생각할 수 있음



In [10]:
pd.DataFrame({'a':[1,2,3], 'b':[2,3,4]})

,a,b
0,1,2
1,2,3
2,3,4


In [11]:
pd.DataFrame(d.to_dict())

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2


In [12]:
d.to_pandas()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2


# 쉬운 함수들

## A. `.select()`

표본 중 일부분을 선택함

특정 구간을 셀렉하여 리스트로 표현함

### 예시1

In [13]:
#d = datasets.Dataset.from_list([emotion['train'][0],emotion['train'][1],emotion['train'][2],emotion['train'][3]])
d = emotion['train'].select(range(4))  # 위와 동일한 코드
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

## B. `.shuffle()`

### 예시1

In [14]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [15]:
show(d.shuffle())
# 랜덤으로 표본 추출

List Overview:
Total items: 4

1. list[0]
   - Type: dict
   - Length: 2
   - Values: {'text': 'i didnt feel humiliated', 'label': 0}

2. list[1]
   - Type: dict
   - Length: 2
   - Values: {'text': 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property', 'label': 2}

3. list[2]
   - Type: dict
   - Length: 2
   - Values: {'text': 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'label': 0}

4. list[3]
   - Type: dict
   - Length: 2
   - Values: {'text': 'im grabbing a minute to post i feel greedy wrong', 'label': 3}


### 예시2

In [16]:
# emotion['train']에서 "처음" 4개의 observation/example 을 뽑는 코드
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [17]:
# emotion['train']에서 '랜덤으로" 4개의 observation/example 을 뽑는 코드
d = emotion['train'].shuffle().select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

## C. `.select_columns()`

`.select_columns()` 함수의 입력값은 `str` 또는 `list`로 받음

### 예시1

In [18]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [19]:
d.select_columns(['text']) # 입력값은 str 또는 list로 받음

Dataset({
    features: ['text'],
    num_rows: 4
})

In [20]:
d.select_columns(['label'])

Dataset({
    features: ['label'],
    num_rows: 4
})

In [21]:
d.select_columns(['text','label'])

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

## D. `.set_format()`

`type`을 지정할 수 있음. 여기서는 `pandas`를 입력함

### 예시1

In [22]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [23]:
d.set_format(type="pandas")

In [24]:
# 기존의 d['text'] 출력값
d['text']

,text
0,i didnt feel humiliated
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...


In [25]:
# pandas로 형식이 변환된 d['text'] 출력값
d['text']

,text
0,i didnt feel humiliated
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...


In [26]:
d['label']

,label
0,0
1,0
2,3
3,2


### 예시2

In [27]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [28]:
# label 항목만 pandas로 바꾸고 싶어
d.set_format(type="pandas",columns=['label'])

In [29]:
d['text']

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property']

In [30]:
d['label']

,label
0,0
1,0
2,3
3,2


### 예시3

type = "pt" 로 한다면, `label`의 타입은 `pytorch.tensor`가 된다

In [31]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [32]:
d.set_format(type="pt",columns=['label'])

In [33]:
d['text']

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property']

In [34]:
d['label']

tensor([0, 0, 3, 2])

## E. `.reset_format()`

### 예시1

In [35]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [36]:
d.set_format(type="pt",columns=['label'])

In [37]:
d['text']

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property']

In [38]:
d['label']

tensor([0, 0, 3, 2])

In [39]:
d.reset_format()

In [40]:
d['text']

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property']

In [41]:
d['label']

[0, 0, 3, 2]

# `.map()`

## A. `d.map()`

In [42]:
def m_transform(example):
    # 여기서 `example`은 d[뭐시기]...를 의미
    # example = l = {'text': 'i didnt feel humiliated', 'label': 0}    <<< d[0] 의 출력값
    result = tokenizer(example['text'])
    return result

In [43]:
# `m_transform` 함수는 tokenizer와 동일한 기능을 수행
m_transform(d[0])

{'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

example의 `text`만 따로 빼와서 `tokenizer` 함수에 넣은 후, `result`를 반환하여라

In [44]:
d2 = d.map(m_transform)
d2[0]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

`d2`에 대한 설명:

`text`와 `label`은 유지되고, m_transform값으로 획득한 새로운 `input_ids`와 `attention_mask`가 새롭게 얻어짐

In [45]:
d2

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4
})

특징1: m_transform()은 입력으로 `example = {'text':xxx, 'label':yyy}` 꼴을 가정한다.

특징2: `.map()`은 변환 전 `dict`와 변환 후 `dict`를 합친다.

In [46]:
# 변환 전 딕셔너리
d[0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [47]:
# 변환 후 딕셔너리
m_transform(d[0])

{'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [48]:
# 둘이 합쳐!
d2[0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

## B. `dd.map()`

딕셔너리가 두 개!

In [49]:
dd = datasets.DatasetDict({
    'train':emotion['train'].select(range(4)),
    'test':emotion['test'].select(range(4)),
})
dd

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
})

In [50]:
# 위에 있던 거 그대로 베껴
def m_transform(example):
    # example = l = {'text': 'i didnt feel humiliated', 'label': 0}
    result = tokenizer(example['text'])
    return result

In [51]:
dd2 = dd.map(m_transform)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [52]:
dd2['train'][0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [53]:
dd2['train'][0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [54]:
dd2['test'][0]

{'text': 'im feeling rather rotten so im not very ambitious right now',
 'label': 0,
 'input_ids': [101,
  10047,
  3110,
  2738,
  11083,
  2061,
  10047,
  2025,
  2200,
  12479,
  2157,
  2085,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [55]:
dd2['test'][:1]

{'text': ['im feeling rather rotten so im not very ambitious right now'],
 'label': [0],
 'input_ids': [[101,
   10047,
   3110,
   2738,
   11083,
   2061,
   10047,
   2025,
   2200,
   12479,
   2157,
   2085,
   102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

## C. `d.map(batch=True)`

In [56]:
d = emotion['train'].select(range(8))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 8
})

In [57]:
# 위에서 복붙해오자
def m_transform(example):
    # example = {'text':xxx, 'label':yyy}
    result = tokenizer(example['text'],padding=True)
    return result

In [58]:
d2 = d.map(m_transform)
d2

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8
})

In [59]:
# 실행 결과: `attention_mask`가 모두 1이 나옴@! 패딩이 정상적으로 이루어지지 않았음!
d2[5]

{'text': 'ive been feeling a little burdened lately wasnt sure why that was',
 'label': 0,
 'input_ids': [101,
  4921,
  2063,
  2042,
  3110,
  1037,
  2210,
  10859,
  2098,
  9906,
  2347,
  2102,
  2469,
  2339,
  2008,
  2001,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [60]:
# `attention_mask`가 모두 1ㅠㅜㅜ;;;;
show(d2[:4])

Dictionary Overview:
Total keys: 4
Keys: ['text', 'label', 'input_ids', 'attention_mask']

1. dict['text']
   - Type: list
   - Length: 4
   - Values: ['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property']
2. dict['label']
   - Type: list
   - Length: 4
   - Values: [0, 0, 3, 2]
3. dict['input_ids']
   - Type: list
   - Length: 4
   - Values: [[101, 1045, 2134, 2102, 2514, 26608, 102], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102], [101, 10047, 9775, 1037, 3371, 2000, 2695, 1045, 2514, 20505, 3308, 102], [101, 1045, 2572, 2412, 3110, 16839, 9080, 12863, 2055, 1996, 13788, 1045, 2097, 2113, 2008, 2009, 2003, 2145, 2006, 1996, 3200, 102]]
4. dict['attention_mask']
  

In [61]:
# 내가 원한 것: 이렇게 0이 나와줘야지!
show(tokenizer(d['text'][:2], padding=True)['input_ids'])

# padding을 해야 tensor화 가능하다.........

List Overview:
Total items: 2

1. list[0]
   - Type: list
   - Length: 23
   - Values: [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

2. list[1]
   - Type: list
   - Length: 23
   - Values: [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]


In [62]:
rprint("d2[:4]['input_ids']") # 우왕 조금 더 굵게 나오넹
show(d2[:4]['input_ids'])

d2[:4]['input_ids']

List Overview:
Total items: 4

1. list[0]
   - Type: list
   - Length: 7
   - Values: [101, 1045, 2134, 2102, 2514, 26608, 102]

2. list[1]
   - Type: list
   - Length: 23
   - Values: [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]

3. list[2]
   - Type: list
   - Length: 12
   - Values: [101, 10047, 9775, 1037, 3371, 2000, 2695, 1045, 2514, 20505, 3308, 102]

4. list[3]
   - Type: list
   - Length: 22
   - Values: [101, 1045, 2572, 2412, 3110, 16839, 9080, 12863, 2055, 1996, 13788, 1045, 2097, 2113, 2008, 2009, 2003, 2145, 2006, 1996, 3200, 102]


**그래서 어떻게 하라고?**

↪`batch`를 사용하세요

```python
# 위에서 복붙해오자
def m_transform(example):
    # example = {'text':xxx, 'label':yyy}
    result = tokenizer(example['text'],padding=True)
    return result

```
아까 쓴 거(오답)

In [63]:
# case1: 위에서 복붙한 버전
# def m_transform(example):
#     # example = {'text':xxx, 'label':yyy}
#     result = tokenizer(example['text'], padding=True)
#     return result

# case2: 의미를 명확하게 작성한 버전
def m_transform_batch(example_batch):
    # example_batch = {'text':[xxx,xxxx], 'label':[yyy,yyyy]}
    result = tokenizer(example_batch['text'], padding=True)
    return result

In [64]:
# 이번에 쓴 거(정답)  // batched=True 및 batch_size=2를 새롭게 넣어
d2 = d.map(m_transform_batch,batched=True,batch_size=2)
d2

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8
})

In [65]:
# 이번에 쓴 거(정답)  // batched=True 및 batch_size=2를 새롭게 넣어
d2 = d.map(m_transform_batch,batched=True,batch_size=2)
d2

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8
})

In [66]:
rprint("d2[:4]['input_ids']")
show(d2[:4]['input_ids'])

d2[:4]['input_ids']

List Overview:
Total items: 4

1. list[0]
   - Type: list
   - Length: 23
   - Values: [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

2. list[1]
   - Type: list
   - Length: 23
   - Values: [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]

3. list[2]
   - Type: list
   - Length: 22
   - Values: [101, 10047, 9775, 1037, 3371, 2000, 2695, 1045, 2514, 20505, 3308, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

4. list[3]
   - Type: list
   - Length: 22
   - Values: [101, 1045, 2572, 2412, 3110, 16839, 9080, 12863, 2055, 1996, 13788, 1045, 2097, 2113, 2008, 2009, 2003, 2145, 2006, 1996, 3200, 102]


`bathed_size`의 default값은 1,000개임

즉, 1,000개가 한 묶음이므로, 2개씩 묶고 싶다면 따로 `bathed_size=원하는숫자(?)`로 작성해야 함

또한, 묶여으니 길이(??)도 짧아짐.

기존 8개에서 2개씩 묶었다면, 길이는 4로 바뀜

In [67]:
# 이번에 쓴 거(정답)  // batched=True 및 batch_size=2를 새롭게 넣어
d2 = d.map(m_transform_batch,batched=True)
d2

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8
})

In [68]:
# 반만 정답. `batch_size`를 입력하지 않았기 때문에 디폴트값이 1,000개로 묶여 `Length`가 동일함
show(d2[:4]['input_ids'])

List Overview:
Total items: 4

1. list[0]
   - Type: list
   - Length: 30
   - Values: [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

2. list[1]
   - Type: list
   - Length: 30
   - Values: [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102, 0, 0, 0, 0, 0, 0, 0]

3. list[2]
   - Type: list
   - Length: 30
   - Values: [101, 10047, 9775, 1037, 3371, 2000, 2695, 1045, 2514, 20505, 3308, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

4. list[3]
   - Type: list
   - Length: 30
   - Values: [101, 1045, 2572, 2412, 3110, 16839, 9080, 12863, 2055, 1996, 13788, 1045, 2097, 2113, 2008, 2009, 2003, 2145, 2006, 1996, 3200, 102, 0, 0, 0, 0, 0, 0, 0, 0]


**`.map(batch=True)`의 특징**

특징1: m_transform_batch()은 입력으로 example_batch = {'text':[xxx,xxxx,...], 'label':[yyy,yyyy,...]} 꼴을 가정한다.

`cf`. example = {'text':xxx, 'label':yyy}

특징2: example_batch는 batch_size만큼 데이터가 있다고 생각한다.

## D. `d.map()` + 칼럼 선택

### 예제1:`attention_mask` 제외

In [69]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

`풀이1`

In [70]:
def m_transform(example):
    # example = {'text':xxx, 'label':yyy}
    result = tokenizer(example['text'])
    return result

In [71]:
d2 = d.map(m_transform)
d2 = d2.select_columns(['text', 'label', 'input_ids'])
d2

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids'],
    num_rows: 4
})

In [72]:
map?

In [73]:
d2[0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102]}

In [74]:
d2[:2]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake'],
 'label': [0, 0],
 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102],
  [101,
   1045,
   2064,
   2175,
   2013,
   3110,
   2061,
   20625,
   2000,
   2061,
   9636,
   17772,
   2074,
   2013,
   2108,
   2105,
   2619,
   2040,
   14977,
   1998,
   2003,
   8300,
   102]]}

`풀이2`

In [75]:
def m_transform(example):
    # example = {'text':xxx, 'label':yyy}
    result = tokenizer(example['text'])
    del result['attention_mask'] # result의 `attention_mask`만! 지워라 == result의 'text', 'label', 'input_ids' 만 남게 됨
    return result

In [76]:
d2 = d.map(m_transform)
d2

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids'],
    num_rows: 4
})

In [77]:
d2[0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102]}

### 예제2: `text` 제외

In [78]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

`풀이1`

In [79]:
def m_transform(example):
    # example = {'text': xxx, 'label':yyy}
    result = tokenizer(example['text'])
    del example['text']
    return result

In [80]:
d2 = d.map(m_transform)
d2

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 4
})

In [81]:
d2[0]

{'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

`풀이2`

In [82]:
def m_transform(example):
    # example = {'text': xxx, 'label':yyy}
    result = tokenizer(example['text'])
    return result

In [83]:
d2 = d.map(m_transform)
d2 = d2.select_columns(['label', 'input_ids', 'attention_mask'])
d2

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 4
})

In [84]:
d2[:1]

{'label': [0],
 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

## `d.map()` + 타입 변환(★)

### 예제1: `.map()`을 통한 타입 변환은 不可

In [85]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

`풀이1` - 실패!

In [86]:
# 왠지 나같아도 이렇게 짰을 것 같은 코드
def m_transform(example):
    # example = {'text': xxx, 'label':yyy}
    result = tokenizer(example['text'])
    del example['text']
    result['input_ids'] = torch.tensor(result['input_ids'])
    result['attention_mask'] = torch.tensor(result['attention_mask'])
    return result

In [87]:
m_transform(d[0])

# 되는 것 같은뎅...

{'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1])}

In [88]:
d2 = d.map(m_transform)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [89]:
d2[0]

{'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [90]:
d2[:1]

{'label': [0],
 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

안 바뀌었네

`tensor`가 없어

`풀이2`: `tokenizer` 이용하기  - 실패

In [91]:
# tokenizer를 사용
def m_transform(example):
    # example = {'text': xxx, 'label':yyy}
    result = tokenizer(example['text'],return_tensors='pt')
    del example['text']
    return result

In [92]:
d[0]['text']

'i didnt feel humiliated'

In [104]:
tokenizer(d[0]['text'],return_tensors='pt')

# `tensor` 나오는 거 보니 되는 것 같은뎅...

{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [94]:
d2 = d.map(m_transform)
d2

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 4
})

In [95]:
d2[0]

{'label': 0,
 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [105]:
d2[:1]

{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]]),
 'label': [0]}

`tensor`가 없어!!

심지어 대괄호가 하나 더 생겼어

`풀이3`: `list`로 만들기 - 실패

일일이 `tensor`로 바꾼 후에 리스트로 묶자!

In [96]:
lst = d.to_list()
lst

[{'text': 'i didnt feel humiliated', 'label': 0},
 {'text': 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'label': 0},
 {'text': 'im grabbing a minute to post i feel greedy wrong', 'label': 3},
 {'text': 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'label': 2}]

In [97]:
lst2 = []
for l in lst:
    result = tokenizer(l['text'])
    result['input_ids'] = torch.tensor(result['input_ids'])
    result['attention_mask'] = torch.tensor(result['attention_mask'])
    del l['text']
    lst2.append(l|result)
lst2

[{'label': 0, 'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1])},
 {'label': 0, 'input_ids': tensor([  101,  1045,  2064,  2175,  2013,  3110,  2061, 20625,  2000,  2061,
          9636, 17772,  2074,  2013,  2108,  2105,  2619,  2040, 14977,  1998,
          2003,  8300,   102]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'label': 3, 'input_ids': tensor([  101, 10047,  9775,  1037,  3371,  2000,  2695,  1045,  2514, 20505,
          3308,   102]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'label': 2, 'input_ids': tensor([  101,  1045,  2572,  2412,  3110, 16839,  9080, 12863,  2055,  1996,
         13788,  1045,  2097,  2113,  2008,  2009,  2003,  2145,  2006,  1996,
          3200,   102]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}]

In [107]:
d2 = datasets.Dataset.from_list(lst2)
d2

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 4
})

In [108]:
d2[0]

{'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [109]:
d2[:1]

{'label': [0],
 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

왜 `tensor`가 없냐고

`풀이4` - 성공?

In [110]:
def m_transform(example):
    # example = {'text': xxx, 'label': yyy}
    result = tokenizer(example['text'])
    del example['text']
    return result

# 여기까지는 할만 하군

In [121]:
d2 = d.map(m_transform)
# 여기까지도 뭐...

d2.set_format(type="pt",columns=['input_ids','attention_mask'],output_all_columns=True)

In [122]:
d2[0]

{'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1]),
 'label': 0}

In [126]:
d2[:1]

{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]]),
 'label': [0]}

됐다~!~!!~!

In [128]:
d2[:2]['input_ids'] # [텐서, 텐서] 형식

[tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]),
 tensor([  101,  1045,  2064,  2175,  2013,  3110,  2061, 20625,  2000,  2061,
          9636, 17772,  2074,  2013,  2108,  2105,  2619,  2040, 14977,  1998,
          2003,  8300,   102])]

두 텐서의 길이가 달라져서 `matrix`형태로 만들지 못 함

`matrix` 형식으로 만들면 좋을 텐데ㅜ

In [130]:
d2[:1]['input_ids'] # 2차원

tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]])

# `.with_transform()`

## A. `d.with_transform()`


### 예제1

In [131]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

`풀이`

In [132]:
dct = d.to_dict()
dct

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property'],
 'label': [0, 0, 3, 2]}

In [136]:
d[0]['text']

'i didnt feel humiliated'

In [137]:
tokenizer(d[0]['text'])

{'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [133]:
result = tokenizer(d['text'])
dct2 = dct | result
d2 = datasets.Dataset.from_dict(dct2)

In [138]:
show(dct2)

Dictionary Overview:
Total keys: 4
Keys: ['text', 'label', 'input_ids', 'attention_mask']

1. dict['text']
   - Type: list
   - Length: 4
   - Values: ['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property']
2. dict['label']
   - Type: list
   - Length: 4
   - Values: [0, 0, 3, 2]
3. dict['input_ids']
   - Type: list
   - Length: 4
   - Values: [[101, 1045, 2134, 2102, 2514, 26608, 102], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102], [101, 10047, 9775, 1037, 3371, 2000, 2695, 1045, 2514, 20505, 3308, 102], [101, 1045, 2572, 2412, 3110, 16839, 9080, 12863, 2055, 1996, 13788, 1045, 2097, 2113, 2008, 2009, 2003, 2145, 2006, 1996, 3200, 102]]
4. dict['attention_mask']
  

In [134]:
d2[0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [135]:
d2[:1]

{'text': ['i didnt feel humiliated'],
 'label': [0],
 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

⇑ 지금까지 배웠던 기능으로만 `d2` 만들기

`풀이2`

In [164]:
def w_transform(examples):
    #examples = {'text':[xxx,xxxx,...], 'label':[yyy,yyyy,...]}  <<< value들이 list로!
    result = tokenizer(examples['text'])
    result = examples | result          # <<< transform은 합쳐주는 과정 必
    return result

⇑ `풀이1`의 내용으로 함수 만들기

`examples` 개념 도입!

In [141]:
d2 = d.with_transform(w_transform)
d2

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [143]:
d2[0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [145]:
d2[:1]

{'text': ['i didnt feel humiliated'], 'label': [0], 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

**`.with_transform()`와 `.map()`의 차이점**

1. `.map()`은 입력으로 `example`꼴을, `.with_transform()`은 입력으로 `examples`를 기대한다.(★)

2. `.map()`은 변환 전과 변환 후 데이터가 자동으로 합쳐진다. `.with_transform()`은 변환 후 데이터만 살아남는다.(★)


3. `.map()`은 변환이 실제로 이루어진다. `.with_transform()`은 변환이 실제로 이루어지지 않다가 d[0],d[:1] 등이 실행하는 순간 이루어진다.

### 예제2

In [147]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

`풀이`

In [148]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,....], 'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'])
    del result['attention_mask']
    result['text'] = examples['text']
    result['label'] = examples['label']
    return result

In [149]:
d2 = d.with_transform(w_transform)
d2

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [150]:
d2[0]

{'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'text': 'i didnt feel humiliated',
 'label': 0}

In [151]:
d2[:1]

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'text': ['i didnt feel humiliated'], 'label': [0]}

### 예제3

In [152]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [153]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,....], 'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'])
    result['label'] = examples['label']
    return result

In [154]:
d2 = d.with_transform(w_transform)
d2

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [155]:
d2[0]

{'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'label': 0}

In [156]:
d2[:1]

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]], 'label': [0]}

### 예제4

In [157]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [166]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,...],'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'])
    result['label'] = torch.tensor(examples['label'])
    result['input_ids'] = torch.tensor(result['input_ids'])
    result['attention_mask'] = torch.tensor(result['attention_mask'])
    return result

In [167]:
d2 = d.with_transform(w_transform)
d2

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [168]:
d2[0]

{'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1]),
 'label': tensor(0)}

In [169]:
d2[:1]

{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]]), 'label': tensor([0])}

In [170]:
d2[:2]

ValueError: expected sequence of length 7 at dim 1 (got 23)

으악!!!!!!!

**에러나는 이유**

`input_ids`의 길이가 다르므로 `torch.tensor(result['input_ids'])` 과정에서 에러 발생


→ `padding`을 해줘야 할 것 같은데?

`풀이2` - 성공!

In [171]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,...],'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'],padding=True)
    result['label'] = torch.tensor(examples['label'])
    result['input_ids'] = torch.tensor(result['input_ids'])
    result['attention_mask'] = torch.tensor(result['attention_mask'])
    return result

In [172]:
d2 = d.with_transform(w_transform)
d2

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [173]:
d2[0]

{'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1]),
 'label': tensor(0)}

In [174]:
d2[:1]

{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]]), 'label': tensor([0])}

In [175]:
d2[:2]

{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1045,  2064,  2175,  2013,  3110,  2061, 20625,  2000,  2061,
          9636, 17772,  2074,  2013,  2108,  2105,  2619,  2040, 14977,  1998,
          2003,  8300,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'label': tensor([0, 0])}

`풀이3`: padding & return_tensor - 이것도 성공

In [176]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,...],'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'],padding=True,return_tensors="pt")
    result['label'] = torch.tensor(examples['label'])
    return result

In [177]:
d2 = d.with_transform(w_transform)
d2

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [178]:
d2[0]

{'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1]),
 'label': tensor(0)}

In [179]:
d2[:1]

{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]]), 'label': tensor([0])}

In [180]:
d2[:2]

{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1045,  2064,  2175,  2013,  3110,  2061, 20625,  2000,  2061,
          9636, 17772,  2074,  2013,  2108,  2105,  2619,  2040, 14977,  1998,
          2003,  8300,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'label': tensor([0, 0])}

## B. `dd.with_transform()`


### 예제1

In [181]:
dd = datasets.DatasetDict({
    'train':emotion['train'].select(range(4)),
    'test':emotion['test'].select(range(4)),
})
dd

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
})

`풀이`

In [182]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,...], 'label':[yyy,yyyy,....]}
    result = tokenizer(examples['text'])
    result = examples | result
    return result

In [183]:
dd2 = dd.with_transform(w_transform)
dd2

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
})

In [184]:
dd2['train'][0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [185]:
dd2['train'][:1]

{'text': ['i didnt feel humiliated'], 'label': [0], 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [187]:
dd2['test'][:1]

{'text': ['im feeling rather rotten so im not very ambitious right now'], 'label': [0], 'input_ids': [[101, 10047, 3110, 2738, 11083, 2061, 10047, 2025, 2200, 12479, 2157, 2085, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

## C. `d.reset_format()`

### 예시1

In [188]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [189]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,...], 'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'])
    return result

In [190]:
d2 = d.with_transform(w_transform)
d2

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [191]:
d2[0]

{'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [192]:
d2[:1]

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [193]:
d2.reset_format()

In [197]:
d2[0] # 다시 d로 돌아왔넹

{'text': 'i didnt feel humiliated', 'label': 0}

In [196]:
d2[:1]

{'text': ['i didnt feel humiliated'], 'label': [0]}

In [198]:
d2 = d.with_transform(w_transform)

뭐 어쨌다는 거지

### 예시2

In [199]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [200]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,...], 'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'],padding=True)
    result['label'] = examples['label']
    return result

In [201]:
d2 = d.with_transform(w_transform)
d2

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [202]:
d2[0]

{'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'label': 0}

In [203]:
d2[:1]

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]], 'label': [0]}

In [204]:
d2[:2]

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'label': [0, 0]}

In [205]:
d2.set_format(type="pt")

In [208]:
d2[0]

# 나는 `attention_mask`와 `input_ids`의 `tensor`를 원했는데 reset 됨??? `label`만 텐서형??

{'text': 'i didnt feel humiliated', 'label': tensor(0)}

In [210]:
d2[:2]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake'],
 'label': tensor([0, 0])}

**왜 그런 거지??**

`.with_transform` 은 `.set_format` 궁합이 안 맞음

`.map()`과 `.set_format` 의 궁합은 굿

# 미묘한 차이

## 예시1

In [211]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [212]:
def m_transform(example):
    # example = {'text':xxx, 'label':yyy}
    result = tokenizer(example['text'])
    return result

In [213]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,...] 'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'])
    result = examples | result
    return result

In [214]:
d2 = d.map(m_transform)
d2.set_format(type='pt')
d2[:1]

{'text': ['i didnt feel humiliated'],
 'label': tensor([0]),
 'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [215]:
d3 = d.with_transform(w_transform)
d3.set_format(type='pt')
d3[:1]

{'text': ['i didnt feel humiliated'], 'label': tensor([0])}

```python
d2 = d.map(m_transform)
d2.set_format(type='pt')
d2[:1]
```

```python
d3 = d.with_transform(w_transform)
d3.set_format(type='pt')
d3[:1]
```
의 출력값이 다르다

후자의 경우, `set_format`을 사용할 경우, `d.with_transform(w_transform)`의 함수가 취소(?)된다

### 예시2

In [216]:
d = emotion['train'].select(range(4))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [217]:
def m_transform(example):
    # example = {'text':xxx, 'label':yyy}
    result = tokenizer(example['text'])
    return result

In [218]:
def w_transform(examples):
    # examples = {'text':[xxx,xxxx,...] 'label':[yyy,yyyy,...]}
    result = tokenizer(examples['text'],return_tensors="pt")
    result = examples | result
    result['label'] = torch.tensor(result['label'])
    return result

In [222]:
d2 = d.map(m_transform)
d2.set_format(type='pt')
d2[:1]

{'text': ['i didnt feel humiliated'],
 'label': tensor([0]),
 'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [220]:
d3 = d.with_transform(w_transform)
d3[:1]

{'text': ['i didnt feel humiliated'], 'label': tensor([0]), 'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

둘 다 똑같아 보이는데

In [224]:
d2 = d.map(m_transform)
d2.set_format(type='pt')
d2[:2]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake'],
 'label': tensor([0, 0]),
 'input_ids': [tensor([  101,  1045,  2134,  2102,  2514, 26608,   102]),
  tensor([  101,  1045,  2064,  2175,  2013,  3110,  2061, 20625,  2000,  2061,
           9636, 17772,  2074,  2013,  2108,  2105,  2619,  2040, 14977,  1998,
           2003,  8300,   102])],
 'attention_mask': [tensor([1, 1, 1, 1, 1, 1, 1]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]}

In [226]:
d3 = d.with_transform(w_transform)
d3[:2]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

또 패딩이야?

↪ 아니야 ㅠ